# Enviroment Setup

In [ ]:
!git clone https://github.com/Kolibri25/WINDIFY_AI


fatal: destination path 'WINDIFY_AI' already exists and is not an empty directory.


In [ ]:
cd WINDIFY_AI

/content/WINDIFY_AI


Setup environment

In [ ]:
!pip install -r requirements.txt

In [ ]:
import scripts.download_barra as dnb
import scripts.load_data as ld
from scripts.model_input import create_uv_tensor, create_uv_tensor_era5, split_tensor


Below purely to document BARRA data download and import procedure:

In [ ]:
'''

# download barrac2 data from months of Oct, Nov, Dec, Jan from 1979 to 2024
# lat/lon values and variables are hard-coded in the script

dnb.main()

# download era5
# TODO: write script to download era5 data

'''

Importing previously downloaded data using gdown from Google Drive folder

In [ ]:

# combine files into single dataset per variable
# BARRA-C2
ld.load_and_save_barra_c2()
# ERA5
ld.load_and_save_era5()

In [1]:
!pip install gdown
!gdown --folder https://drive.google.com/drive/folders/1xwqJ8Ku_US5Q7MRvExKC26TqCUroQ6QE?usp=sharing

Retrieving folder contents
Retrieving folder 1_rHNYO880eYBNB0w7ciNWKxCop2oq4Na ERA5
Retrieving folder 1aMx8fvte7O34QJNiYFbA9InUIqaqYSU4 ERA5_1997_daily
Processing file 150cm8X3zbPjB6vLBfc_lXhz69o_NUv-N 10m_v_component_of_wind_0_daily-mean.nc
Processing file 1X0LPR6Fc0ruAnFUtfCbLd_XdXU6Hj3lw mean_sea_level_pressure_0_daily-mean.nc
Processing file 16uuceEdQIuOa2_DqfBd-NpUtFhJbqKcw 10m_u_component_of_wind_stream-oper_daily-mean.nc
Processing file 1-byITeSOvLZh4_PlLOOXkFpxQLn3Vst7 mean_wave_direction_1_daily-mean.nc
Processing file 14PKWgeUU9ZM9hPPb4PavDz3KjlPT7SL_ 2m_temperature_stream-oper_daily-mean.nc
Retrieving folder 1fxU9eG1R9n6yRAZWRJaDs7PPcHpqGwSi ERA5_2008_daily
Processing file 1la2ORXptnilfGOHCE1uwmHb6ZllnEfso 10m_u_component_of_wind_stream-oper_daily-mean.nc
Processing file 1dROXCeKB5tjrRY68YvJ1cNyoqITl9QV0 mean_wave_direction_1_daily-mean.nc
Processing file 1FgJ8TJ_P9mnixFp5aUKiahZaUuWcGK3w mean_sea_level_pressure_0_daily-mean.nc
Processing file 1DcNBfMAzWykYG015gkh1BHuURxJYe9

In [ ]:
!mv metfut_data/* /content/WINDIFY_AI/data/

In [ ]:
# load target data (BARRA-C2)

uas_ds = xr.open_dataset('/content/WINDIFY_AI/data/BARRA-C2/uas_gesamt.nc')
vas_ds = xr.open_dataset('/content/WINDIFY_AI/data/BARRA-C2/vas_gesamt.nc')

# load input data (ERA5)

era5_ds = xr.open_dataset('/content/WINDIFY_AI/data/ERA5/combined_era5.nc')

# Data Preprocessing

Steps for Target ie. BARRA-C2:

1. Drop grid points to match model dimensions (divisible by 16).
2. Normalise the data.


Steps for Input ie. ERA5

1.   Drop grid points to match model dimensions (divisible by 16).
2.   Regrid EAR5 to a high-resolution grid of BARRA-C2
3.  Normalise the data.


Finally, we stack the input/target together as PyTorch tensors and split the data.

## Target: BARRA-C2

In [ ]:
# crop to make dimensions divisible by 16

from scripts.crop_to_div16 import crop_to_multiple_of_16

uas_ds = crop_to_multiple_of_16(uas_ds, lat_name='lat', lon_name='lon')
vas_ds = crop_to_multiple_of_16(vas_ds, lat_name='lat', lon_name='lon')


In [ ]:
# normalise the target data

from scripts.normalize_data import normalize_dataset

uas_ds, uas_mean, uas_std = normalize_data(uas_ds,"uas")
vas_ds, vas_mean, vas_std = normalize_data(vas_ds,"vas")

## Input: ERA5

In [ ]:
# regrid to barra grid

era5_interp = era5_ds.interp(
    lat=uas_ds.lat,
    lon=uas_ds.lon,
    method='nearest'
)

In [ ]:
# normalise input data

u10_ds, u10_mean, u10_std = normalize_data(era5_interp,"u10")["u10"]
v10_ds, v10_mean, v10_std = normalize_data(era5_interp,"v10")["v10"]

In [ ]:
# convert to PyTorch tensors

target = mi.create_uv_tensor(uas_ds,vas_ds)
input = mi.create_uv_tensor_era5(u10_ds,v10_ds)


In [ ]:
# check shapes

print(target.shape)
print(input.shape)

In [ ]:
# save in case of runtime shortage

torch.save(target, 'target.pt')
torch.save(input, 'input.pt')



In [ ]:
# train/validation/test split

x_train, x_val, x_test = split_tensor(x_tensor)
y_train, y_val, y_test = split_tensor(y_tensor)